In [3]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 1.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 747.3 kB/s eta 0:00:00m eta 0:00:010:01:01


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

/home/ia/miniconda3/envs/pytorch_musika/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./mergedfeaturesnonormalization.csv',low_memory=False)

In [3]:
df.drop(columns=['Unnamed: 0','id','name','artist','genre','popularity','lyricsBillboard','lyric','language'], inplace=True)

In [4]:
df.head()

,rms,spec_centroids,spec_bw,spec_flatness,spec_rolloff,zero_crossing_rate,spec_contrast_0,spec_contrast_1,spec_contrast_2,spec_contrast_3,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.201473,1459.794079,2347.101930,0.000422,2703.750874,0.031043,21.093848,11.112248,16.837817,18.570924,...,-9.142,1.0,0.0599,0.84800,0.477000,0.3960,0.655,125.068,134442.0,4
1,0.241160,2794.821171,3049.131322,0.001429,5929.202261,0.065948,17.227330,11.030906,14.381297,16.193530,...,-5.708,0.0,0.0286,0.00664,0.000011,0.1680,0.543,99.009,233933.0,4
2,0.157690,3197.043081,3562.309993,0.002606,6817.953306,0.071609,15.384177,11.536696,15.757186,17.599652,...,-5.946,0.0,0.2690,0.14300,0.000000,0.0771,0.613,94.948,270507.0,4
3,0.251467,4104.010298,4402.199880,0.003410,9487.646250,0.074107,18.610099,12.542007,17.452827,17.833720,...,-4.582,0.0,0.2530,0.23300,0.000000,0.4370,0.778,156.932,244533.0,4
4,0.173395,3598.760089,3651.350338,0.003948,7642.456367,0.095362,14.025943,11.345112,15.192431,16.994633,...,-4.913,0.0,0.0447,0.30100,0.000025,0.2130,0.846,95.058,212613.0,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30672 entries, 0 to 30671
Columns: 194 entries, rms to time_signature
dtypes: float64(193), int64(1)
memory usage: 45.4 MB


In [6]:
X = df.loc[:, df.columns != 'hit']
y = df.hit

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
X_train.shape

(24537, 193)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [14]:
from torch.nn.modules.loss import MSELoss
import os
from time import process_time
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCHSIZE = 1024
INPUT_SIZE = 193
CLASSES = 193
DIR = os.getcwd()
EPOCHS = 50
N_TRIALS = 500
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

train_tensor = torch.tensor(X_train)
test_tensor = torch.tensor(X_test)


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 2, 6)
    layers = []
    autoencoder = nn.Sequential()
    name_layers = list()
    activationFunc = nn.ReLU()

    in_features = 193
    #Encoder
    for i in range(n_layers,0,-1):
        out_features = int((CLASSES/(2**2))*i**3)
        autoencoder.add_module("encoder_layer{}".format(i),nn.Linear(in_features, out_features))
        trial.set_user_attr("encoder_layer{}".format(i),[in_features, out_features])
        activations = [nn.ReLU(),nn.GELU(),nn.SELU(),nn.LeakyReLU(),nn.ELU(),nn.Tanh()]
        num_activation = trial.suggest_categorical("activation", [0,1,2,3,4,5])
        activationFunc = activations[num_activation]
        autoencoder.add_module("encoder_activation{}".format(i),activationFunc)
        #print("encoder",in_features,out_features)
        #layers.append(nn.Linear(in_features, out_features))
        #layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.0, 0.5)
        #layers.append(nn.Dropout(p))
        autoencoder.add_module("encoder_dropout{}".format(i),nn.Dropout(p))
        trial.set_user_attr("encoder_dropout{}".format(i),p)
        in_features = out_features

    #Bottleneck
    bottleneck = trial.suggest_int("bottleneck_size", 32, 128)
    autoencoder.add_module("bottleneck",nn.Linear(in_features,bottleneck))
    trial.set_user_attr("bottleneck",[in_features,bottleneck])
    #print("bottleneck",in_features,bottleneck)
    autoencoder.add_module("sigmoid",nn.Sigmoid())
    #layers.append(nn.Linear(in_features,bottleneck))
    #layers.append(nn.Sigmoid())

  
    for i in range(1,n_layers+1):
        out_features = int((CLASSES/(2**2))*i**3)
        if i == 1:
          #print("decoder",bottleneck,out_features)
          autoencoder.add_module("decoder_layer{}".format(i),nn.Linear(bottleneck, out_features))
          trial.set_user_attr("decoder_layer{}".format(i),[bottleneck, out_features])
        else:
          autoencoder.add_module("decoder_layer{}".format(i),nn.Linear(in_features, out_features))
          trial.set_user_attr("decoder_layer{}".format(i),[in_features, out_features])
          #print("decoder",in_features,out_features)
        
        autoencoder.add_module("decoder_activation{}".format(i),activationFunc)
        #layers.append(nn.Linear(out_features, out_features))
        #layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.0, 0.5)
        autoencoder.add_module("decoder_dropout{}".format(i),nn.Dropout(p))
        trial.set_user_attr("decoder_dropout{}".format(i),p)
        #layers.append(nn.Dropout(p))

        in_features = out_features

    autoencoder.add_module("decoder_final",nn.Linear(in_features, INPUT_SIZE))
    #print("decoder",in_features, INPUT_SIZE)
    trial.set_user_attr("decoder_final",[in_features, INPUT_SIZE])
    #layers.append(nn.Linear(in_features, INPUT_SIZE))

    return autoencoder

def objective(trial):
    # Generate the model.
    model = define_model(trial)
    model= nn.DataParallel(model)
    model.to(device)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD","Adagrad"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
        
    train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=BATCHSIZE, shuffle=True)
    #losses = [nn.MSELoss(),nn.HingeEmbeddingLoss(),nn.CrossEntropyLoss(),]
    #num_loss = trial.suggest_categorical("loss", [0,1,2])
    criterion = nn.MSELoss()
    loss_epoch = []
    for epoch in range(EPOCHS):
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            recon_data = model(data.float())
            loss = criterion(recon_data, data.float())
            loss.backward()
            optimizer.step()      
        loss_epoch.append(loss.item())
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, EPOCHS, loss.item()))
        final_loss = loss_epoch[-1]
        trial.report(final_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return final_loss


chroma_t_s = process_time()
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=N_TRIALS)
chroma_t_e = process_time()
print("Best trial:")
trial = study.best_trial
print(trial)
times = (chroma_t_e-chroma_t_s)
print('Tiempo de ejecucion',times)


[I 2023-05-09 09:19:12,422] A new study created in memory with name: no-name-430fc213-7cdd-4de5-a9f7-7b75481a58b3


Epoch [1/50], Loss: 0.6075
Epoch [2/50], Loss: 0.5100
Epoch [3/50], Loss: 0.4750
Epoch [4/50], Loss: 0.4947
Epoch [5/50], Loss: 0.4205
Epoch [6/50], Loss: 0.3997
Epoch [7/50], Loss: 0.4122
Epoch [8/50], Loss: 0.3899
Epoch [9/50], Loss: 0.3958
Epoch [10/50], Loss: 0.3872
Epoch [11/50], Loss: 0.3769
Epoch [12/50], Loss: 0.3917
Epoch [13/50], Loss: 0.3545
Epoch [14/50], Loss: 0.3715
Epoch [15/50], Loss: 0.4144
Epoch [16/50], Loss: 0.3751
Epoch [17/50], Loss: 0.3747
Epoch [18/50], Loss: 0.3558
Epoch [19/50], Loss: 0.3618
Epoch [20/50], Loss: 0.3556
Epoch [21/50], Loss: 0.3856
Epoch [22/50], Loss: 0.3461
Epoch [23/50], Loss: 0.3352
Epoch [24/50], Loss: 0.3453
Epoch [25/50], Loss: 0.3516
Epoch [26/50], Loss: 0.3381
Epoch [27/50], Loss: 0.3412
Epoch [28/50], Loss: 0.3515
Epoch [29/50], Loss: 0.3268
Epoch [30/50], Loss: 0.3497
Epoch [31/50], Loss: 0.3303
Epoch [32/50], Loss: 0.3464
Epoch [33/50], Loss: 0.3337
Epoch [34/50], Loss: 0.3472
Epoch [35/50], Loss: 0.3338
Epoch [36/50], Loss: 0.3297
E

[I 2023-05-09 09:19:23,590] Trial 0 finished with value: 0.3159120976924896 and parameters: {'n_layers': 2, 'activation': 3, 'dropout_l2': 0.06448507264341685, 'dropout_l1': 0.16324666757183226, 'bottleneck_size': 68, 'optimizer': 'Adam', 'lr': 0.007413547583016127}. Best is trial 0 with value: 0.3159120976924896.


Epoch [50/50], Loss: 0.3159
Epoch [1/50], Loss: 0.9335
Epoch [2/50], Loss: 0.9891
Epoch [3/50], Loss: 1.0219
Epoch [4/50], Loss: 1.0077
Epoch [5/50], Loss: 1.0178
Epoch [6/50], Loss: 0.9843
Epoch [7/50], Loss: 1.0553
Epoch [8/50], Loss: 0.9850
Epoch [9/50], Loss: 0.9869
Epoch [10/50], Loss: 0.9824
Epoch [11/50], Loss: 0.9719
Epoch [12/50], Loss: 0.9589
Epoch [13/50], Loss: 1.0264
Epoch [14/50], Loss: 0.9836
Epoch [15/50], Loss: 0.9596
Epoch [16/50], Loss: 1.0336
Epoch [17/50], Loss: 0.9867
Epoch [18/50], Loss: 1.0127
Epoch [19/50], Loss: 1.0288
Epoch [20/50], Loss: 1.0002
Epoch [21/50], Loss: 0.9780
Epoch [22/50], Loss: 0.9665
Epoch [23/50], Loss: 0.9881
Epoch [24/50], Loss: 0.9893
Epoch [25/50], Loss: 0.9892
Epoch [26/50], Loss: 1.0321
Epoch [27/50], Loss: 0.9604
Epoch [28/50], Loss: 1.0100
Epoch [29/50], Loss: 1.0305
Epoch [30/50], Loss: 1.0216
Epoch [31/50], Loss: 0.9749
Epoch [32/50], Loss: 0.9668
Epoch [33/50], Loss: 1.0110
Epoch [34/50], Loss: 1.0140
Epoch [35/50], Loss: 1.0980
E

[I 2023-05-09 09:21:18,342] Trial 1 finished with value: 0.9820969104766846 and parameters: {'n_layers': 5, 'activation': 0, 'dropout_l5': 0.39135440730626125, 'dropout_l4': 0.24077675566796763, 'dropout_l3': 0.44411013230120333, 'dropout_l2': 0.25434840623344906, 'dropout_l1': 0.091971381464306, 'bottleneck_size': 44, 'optimizer': 'Adam', 'lr': 0.00979268027885116}. Best is trial 0 with value: 0.3159120976924896.


...

Epoch [1/50], Loss: 0.5363


[I 2023-05-09 10:05:11,594] Trial 498 pruned. 


Epoch [1/50], Loss: 0.5341


[I 2023-05-09 10:05:12,024] Trial 499 pruned. 


Epoch [1/50], Loss: 0.5466
Best trial:
FrozenTrial(number=113, state=TrialState.COMPLETE, values=[0.08030945807695389], datetime_start=datetime.datetime(2023, 5, 9, 9, 52, 52, 654348), datetime_complete=datetime.datetime(2023, 5, 9, 9, 53, 3, 859817), params={'n_layers': 3, 'activation': 2, 'dropout_l3': 0.02386353738159614, 'dropout_l2': 0.0177066124034864, 'dropout_l1': 6.388088319771168e-05, 'bottleneck_size': 112, 'optimizer': 'Adam', 'lr': 0.0008964600124897553}, user_attrs={'encoder_layer3': [193, 1302], 'encoder_dropout3': 0.02386353738159614, 'encoder_layer2': [1302, 386], 'encoder_dropout2': 0.0177066124034864, 'encoder_layer1': [386, 48], 'encoder_dropout1': 6.388088319771168e-05, 'bottleneck': [48, 112], 'decoder_layer1': [112, 48], 'decoder_dropout1': 6.388088319771168e-05, 'decoder_layer2': [48, 386], 'decoder_dropout2': 0.0177066124034864, 'decoder_layer3': [386, 1302], 'decoder_dropout3': 0.02386353738159614, 'decoder_final': [1302, 193]}, system_attrs={}, intermediate_v

In [15]:
print(f'tiempo de ejecucion {times}')

tiempo de ejecucion 2983.2258980620027


In [13]:
##tiempo de ejecucion 32621.433589187 200 trials

In [ ]:
# tiem´o de ejecucion 2983.2258980620027 500 trials

In [34]:
num_df = 2
trial_df = study.trials_dataframe()
trial_df.to_csv('./trial_df{}.csv'.format(num_df))

In [37]:
study.trials[222].user_attrs

{'encoder_layer3': [193, 1302],
 'encoder_dropout3': 0.1528129852902385,
 'encoder_layer2': [1302, 386],
 'encoder_dropout2': 0.1370017411821048,
 'encoder_layer1': [386, 48],
 'encoder_dropout1': 0.00014226668033649227,
 'bottleneck': [48, 57],
 'decoder_layer1': [57, 48],
 'decoder_dropout1': 0.00014226668033649227,
 'decoder_layer2': [48, 386],
 'decoder_dropout2': 0.1370017411821048,
 'decoder_layer3': [386, 1302],
 'decoder_dropout3': 0.1528129852902385,
 'decoder_final': [1302, 193]}

In [36]:
trial_df.head()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_bottleneck_size,params_dropout_l1,params_dropout_l2,params_dropout_l3,...,user_attrs_encoder_dropout4,user_attrs_encoder_dropout5,user_attrs_encoder_dropout6,user_attrs_encoder_layer1,user_attrs_encoder_layer2,user_attrs_encoder_layer3,user_attrs_encoder_layer4,user_attrs_encoder_layer5,user_attrs_encoder_layer6,state
0,0,1.095714,2023-04-05 15:56:48.729028,2023-04-05 16:03:17.103191,0 days 00:06:28.374163,0,97,0.373090,0.419342,0.463378,...,0.118223,0.462644,0.357541,"[386, 48]","[1302, 386]","[3088, 1302]","[6031, 3088]","[10422, 6031]","[193, 10422]",COMPLETE
1,1,0.969105,2023-04-05 16:03:17.104349,2023-04-05 16:03:42.618581,0 days 00:00:25.514232,3,91,0.358028,0.358582,0.268583,...,0.257853,NaN,NaN,"[386, 48]","[1302, 386]","[3088, 1302]","[193, 3088]",NaN,NaN,COMPLETE
2,2,1.242917,2023-04-05 16:03:42.619693,2023-04-05 16:05:31.952431,0 days 00:01:49.332738,1,119,0.464029,0.082718,0.457215,...,0.002015,0.153986,NaN,"[386, 48]","[1302, 386]","[3088, 1302]","[6031, 3088]","[193, 6031]",NaN,COMPLETE
3,3,0.785448,2023-04-05 16:05:31.953757,2023-04-05 16:05:56.413413,0 days 00:00:24.459656,2,33,0.477158,0.414451,0.004463,...,0.055156,NaN,NaN,"[386, 48]","[1302, 386]","[3088, 1302]","[193, 3088]",NaN,NaN,COMPLETE
4,4,4.528540,2023-04-05 16:05:56.414717,2023-04-05 16:07:45.664597,0 days 00:01:49.249880,2,45,0.155759,0.270505,0.080408,...,0.473117,0.103223,NaN,"[386, 48]","[1302, 386]","[3088, 1302]","[6031, 3088]","[193, 6031]",NaN,COMPLETE
